In [12]:
#---PACKAGES---
import pandas as pd
from json import loads, dumps
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [13]:
# Dataset
df = pd.read_csv('complaints.csv')
df = df.dropna()
documents = loads(df.to_json(orient = 'records'))

In [14]:
# QDRANT connection
qdrant_client = QdrantClient(
    url="https://4cc46006-16b3-4d25-b4ba-3819cd7d5f1e.eu-central-1-0.aws.cloud.qdrant.io:6333", 
    api_key="JtMb4VZQsdWStzO38BMRh-qp6PTnEcmzDzGHE_Vh0vSZXewMhIFpNA",
)

In [15]:
# Encoder model for embedding
encoder = SentenceTransformer('all-MiniLM-L6-v2') 

In [16]:
# Creating collection (database)
qdrant_client.recreate_collection(
	collection_name = "comcat-complaints",
	vectors_config = models.VectorParams(
		size = encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
		distance = models.Distance.COSINE
	)
)

True

In [17]:
qdrant_client.upload_records(
	collection_name="comcat-complaints",
	records=[
		models.Record(
			id = idx,
			vector = encoder.encode(doc["text"]).tolist(),
			payload = doc
		) for idx, doc in enumerate(documents)
	]
)

In [45]:
hits = qdrant_client.search(
	collection_name="comcat-complaints",
	query_vector=encoder.encode("very good").tolist(),
	limit=1
)

In [46]:
hits[0].payload['text']

'Great service.. Rarely down. Great speed options.'